In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# IMDB 데이터 셋

### 데이터셋 정보
- 훈련 데이터와 테스트 데이터가 각각 25000개
- 데이터의 각 review에는 label이 붙어 있다.
- 부정은 0, 긍정은 1로 나타낸다.
- 약 50%는 긍정, 50%는 부정 리뷰로 구성
- review 문장의 단어들을 출현빈도순으로 정렬해서 정수로 변환시킨 시퀀스를 x로 한다.
- 스탠포드 대학의 앤드류 마스가 수집했다.

### 예측하고자 하는 방법
- 이진 분류 문제로, 학습 데이터를 이용해, 설계한 신경망을 학습시킨다.
- 학습된 신경망을 이용해 어떤 리뷰가 긍정일 확률을 예측할 수 있도록 한다.

### 학습을 위해 데이터가 어떻게 가공/처리 되었는지?
- 25000개의 샘플을 훈련셋 20000개와 5000개의 검증셋으로 분리
- 길이를 맞추기 위해 패딩 삽입
- 임베딩 층을 통과하여 실수 텐서로 매핑
- Embedding층, conv1D층, Dropout층, LSTM, Dense층을 차례로 통과하며 모델 훈련

### 사용된 모델의 입력층과 출력층
- 입력층
    - 숫자 인덱스를 밀집 벡터로 매핑하는 embedding층
    - (샘플, 시퀀스길이)를 입력 인자로 받음
    - 크기가 (샘플, 시퀀스 길이, 임베딩 차원)인 3D 실수 텐서를 출력함.
- 출력층
    - 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 출력층 dense, 은닉 유닛은 1
    - 확률(0과 1 사이의 점수, 1에 가까울수록 긍정, 0에 가까울 수록 부정)을 출력하기 위해 시그모이드 활성화 함수(로지스틱 함수, vanishing gradient 문제가 있어서 현재는 잘 사용하지 않음)를 사용

###  사용된 모델의 특징에 대한 기술 
- rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 컴파일.
- dropout층을 사용하여 과대적합 방지
- conv1D층의 출력을 maxpooling층을 통과하며 feature 벡터를 줄임.
- LSTM 층으로 모델을 구성하여 기존의 완전연결 네트워크보다 나은 결과를 기대.

In [10]:
from keras.datasets import imdb
import numpy as np

max_features = 10000
text_max_words = 500

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# restore np.load for future normal usage
np.load = np_load_old
#예제 실행시 오류나서 스택오버플로우사이트 참고했습니다.
#https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555

#### 신경망 모델 제작

In [11]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=text_max_words))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(layers.MaxPooling1D(pool_size=4))
model.add(layers.LSTM(128))
model.add(layers.Dense(1, activation='sigmoid'))

In [12]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
from keras.preprocessing import sequence

x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

x_train = sequence.pad_sequences(x_train, maxlen=text_max_words)
x_val = sequence.pad_sequences(x_val, maxlen=text_max_words)
x_test = sequence.pad_sequences(x_test, maxlen=text_max_words)

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=25,
                    batch_size=64,
                    validation_data=(x_val, y_val))
model.summary()